Use the precalculated output from resnet50 and train a Dense (FC) model

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import bcolz

from tensorflow.contrib.keras import layers
from tensorflow.contrib.keras import models
from tensorflow.contrib.keras import optimizers
from tensorflow.contrib.keras import applications

In [3]:
def load_array(fname):
    return bcolz.open(fname)[:]

In [4]:
train_conv_layers = load_array('./data/resnet50/train_conv.arr')
valid_conv_layers = load_array('./data/resnet50/valid_conv.arr')
train_labels = load_array('./data/resnet50/train_labels.arr')
valid_labels = load_array('./data/resnet50/valid_labels.arr')

In [ ]:
train_conv_layers.shape, train_labels.shape

In [ ]:
valid_conv_layers.shape, valid_labels.shape,

In [12]:
inp = layers.Input(shape=(2048,), dtype='float32')

In [13]:
p = 0.6
x = layers.Dense(512, activation='relu')(inp)
x = layers.BatchNormalization()(x)
x = layers.Dropout(p)(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(p / 2)(x)
x = layers.Dense(8, activation='softmax')(x)

In [14]:
model = models.Model(inp, x)

In [15]:
model.compile(optimizers.Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_conv_layers, train_labels, validation_data=(valid_conv_layers, valid_labels), batch_size=64, epochs=10) 

After 10 epochs we got `loss: 0.0693 - acc: 0.9779 - val_loss: 0.3008 - val_acc: 0.9269`

## Submission

In [12]:
import submission

In [14]:
test_conv_layers = load_array('./data/resnet50/test_conv.arr')

In [17]:
test_filenames = load_array('./data/resnet50/test_filenames.arr')

In [26]:
preds = model.predict(test_conv_layers)

In [27]:
submission.gen_file(preds=preds, filenames=test_filenames, clip=True)

This gave a score of :
- public: 1.51250 - not really that good but, just barely above the sample submission but only 325 people were above it
- private: 2.63947 - kinda bad to be honest

Running more 20 more epochs it got public: 1.44111 - private: 2.50299 - which i guess is decent for not doing much work